In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
import autokeras as ak
import numpy as np

Using TensorFlow backend


In [2]:
model = tensorflow.keras.models.load_model('./structured_data_classifier/best_model')
model.summary()

2023-10-28 15:28:47.933531: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-28 15:28:47.933978: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-28 15:28:47.933991: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-28 15:28:47.934281: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-28 15:28:47.934833: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-10-28 15:29:27.887708: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1776)]            0         
                                                                 
 multi_category_encoding (M  (None, 1776)              0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 1776)              3553      
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                56864     
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

In [8]:
# Load training dataset
df1 = pd.read_csv('./bioresponse/train.csv')
#df1.describe() # Visualize

# Load testing dataset
kaggle_test = pd.read_csv('./bioresponse/test.csv')
# test.describe() # Visualize

# Separate labels from training dataset for fitting
labels = df1['Activity']
del df1['Activity']

# 80/20 train test split
x_train, x_test, y_train, y_test = train_test_split(df1, labels, test_size=0.2)
input_shape = x_train.shape
print("Shape of Input Data: " + str(input_shape))
print("Shape of Test Labels: " + str(y_test.shape))

Shape of Input Data: (3000, 1776)
Shape of Test Labels: (751,)


In [10]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)

history = model.fit(
    x_train, y_train, epochs=10, validation_data=(x_test, y_test)
)

Epoch 1/10
94/94 [==============================] - 93s 821ms/step - loss: 27.5366 - accuracy: 0.7557 - val_loss: 15.2058 - val_accuracy: 0.7909
Epoch 2/10
94/94 [==============================] - 48s 506ms/step - loss: 11.2871 - accuracy: 0.8130 - val_loss: 13.3200 - val_accuracy: 0.7936
Epoch 3/10
94/94 [==============================] - 43s 457ms/step - loss: 7.9005 - accuracy: 0.8157 - val_loss: 9.5511 - val_accuracy: 0.8056
Epoch 4/10
94/94 [==============================] - 41s 432ms/step - loss: 6.0338 - accuracy: 0.8210 - val_loss: 8.4378 - val_accuracy: 0.8003
Epoch 5/10
94/94 [==============================] - 42s 444ms/step - loss: 5.4995 - accuracy: 0.8167 - val_loss: 8.0831 - val_accuracy: 0.8003
Epoch 6/10
94/94 [==============================] - 44s 461ms/step - loss: 4.7774 - accuracy: 0.8220 - val_loss: 7.8123 - val_accuracy: 0.8096
Epoch 7/10
94/94 [==============================] - 39s 415ms/step - loss: 4.0045 - accuracy: 0.8277 - val_loss: 7.0756 - val_accuracy: 0.

In [ ]:
prob_output = model.predict(kaggle_test)
print(prob_output)
print(prob_output.shape)
print(kaggle_test.shape)
np.savetxt("my_data.csv", prob_output, delimiter=",")